# Dependences

In [1]:
include("../../src/struct_data.jl")

Aggregate

# Force Functions

## Cubic

In [2]:
@make_struct_func Cubic
dump(Cubic)

UnionAll
  var: TypeVar
    name: Symbol T
    lb: Union{}
    ub: Any
  body: Cubic{T} <: ForceType
    μ₁::T
    rₘᵢₙ::T
    rₘₐₓ::T


In [3]:
# Parameters
μ₁   = 1.0
rₘₐₓ = 2.0
rₘᵢₙ = 3.0

3.0

In [4]:
Cubic(μ₁,rₘₐₓ,rₘᵢₙ)

Cubic{Float64}(1.0, 2.0, 3.0)

In [5]:
# Show function on specific r value
r = 1
force_func(Cubic(μ₁,rₘₐₓ,rₘᵢₙ),r)

4.0

## LennardJones

In [6]:
@make_struct_func LennardJones
dump(LennardJones)

UnionAll
  var: TypeVar
    name: Symbol T
    lb: Union{}
    ub: Any
  body: LennardJones{T} <: ForceType
    μ₁::T
    rₘᵢₙ::T
    rₘₐₓ::T


In [7]:
# Parameters
μ₁   = 1.0
rₘₐₓ = 2.0
rₘᵢₙ = 3.0

3.0

In [8]:
LennardJones(μ₁,rₘₐₓ,rₘᵢₙ)

LennardJones{Float64}(1.0, 2.0, 3.0)

In [9]:
# Show function on specific r value
r = 1.9
force_func(LennardJones(μ₁,rₘₐₓ,rₘᵢₙ),r)

1.9609746568876094

# Model Parameter Struct

In [42]:
ModelPar(
    TimePar(
        t_f      = 10000.0,
        dt       = 0.5
    ),
    NeighborPar(
        n_knn    = 50
    ),
    GeometryPar(
        r_agg       = 15,
        position    = [0 0 0;
                       100 0 0],
        outer_ratio = 0.8
    ),
    SimulationPar(
        n_text = 100,
        path_input = "AAA",
        path_ouput = "AAA",
        name_cell = "Queso"
    )
)

ModelPar(TimePar(10000.0, 0.5), NeighborPar(50), GeometryPar(15.0, [0 0 0; 100 0 0], 0.8), SimulationPar(100, "AAA", "AAA", "Queso"))

# Aggregate

In [43]:
Model = ModelPar(
    TimePar(
        t_f      = 10000.0,
        dt       = 0.5
    ),
    NeighborPar(n_knn = 50),
    GeometryPar(
        r_agg    = 2.0,
        position = [0 0 0;
                    -1 0 0],
        outer_ratio = 0.7
    ),
    SimulationPar(
        n_text     = 100,
        path_input = "../../data/init/Sphere",
        path_ouput = "AAA",
        name_cell  = "Queso"
    )
)

@time AGG = Aggregate(1.0, Model, Cubic(1,2,3), ContractilePar(1))
println("===== Agg Position Matrix ======")
display(AGG.Position.X)
println("===== Position Matrix Size =====")
println("X          = $(size(AGG.Position.X))")
println("dX         = $(size(AGG.Position.dX))")
println("===== Neighbors Matrix Size ====")
println("idx        = $(size(AGG.Neighbor.idx))")
println("idx_red    = $(size(AGG.Neighbor.idx_red))")
println("idx_sum    = $(size(AGG.Neighbor.idx_sum))")
println("idx_cont   = $(size(AGG.Neighbor.idx_sum))")
println("======= Force Matrix Size =======")
println("F          = $(size(AGG.Force.F))")
println("======= Index Aggregate =========")
display(AGG.Index.IdxAgg')
println("===== Outer vs Inner Cells =====")
display(AGG.Index.Outline')
println("Outer/Total = $(sum(AGG.Index.Outline)/size(AGG.Position.X,1))")

  0.000826 seconds (185 allocations: 50.188 KiB)
===== Agg Position Matrix ======


12×3 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 -1.5  -0.58  -0.82
  0.5  -0.58  -0.82
 -0.5   1.15  -0.82
 -0.5  -1.15   0.82
 -1.5   0.58   0.82
  0.5   0.58   0.82
 -2.5  -0.58  -0.82
 -0.5  -0.58  -0.82
 -1.5   1.15  -0.82
 -1.5  -1.15   0.82
 -2.5   0.58   0.82
 -0.5   0.58   0.82

===== Position Matrix Size =====
X          = (12, 3)
dX         = (12, 3)
===== Neighbors Matrix Size ====
idx        = (12, 12)
idx_red    = (21, 12)
idx_sum    = (1, 12)
idx_cont   = (1, 12)
======= Force Matrix Size =======
F          = (12, 3)
======= Index Aggregate =========


1×12 adjoint(::Matrix{Int64}) with eltype Int64:
 1  1  1  1  1  1  2  2  2  2  2  2

===== Outer vs Inner Cells =====


1×12 adjoint(::Vector{Int64}) with eltype Int64:
 1  0  1  1  1  0  1  0  1  1  1  0

Outer/Total = 0.6666666666666666
